<a href="https://colab.research.google.com/github/deveshpatwa/Codsoft/blob/main/IOTA_Attendance_project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# importing numpy and pandas
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# first working on user id data

user_data = open("user.dat","r").read()

In [7]:
# this is the main file which has the attendance data and reading it in csv

df = pd.read_csv("attendance.dat",header=None)

In [3]:
# the file contain non ACSII and symboles that we dont need we are only keeping the data with numbers and char

a = ""
for i in user_data:
    if i.isalnum():
        a += i
    else:
        a += " "
a = a.split()

# variable a contain a list of name,id,name,id,name,id so we need 2 list name and id
name = [a[i] for i in range(len(a)) if i%2==0]
user_id = [int(a[i]) for i in range(len(a)) if i%2==1]

# now converting this 2 list into one dictionary with id as key and name as value
user = dict(zip(user_id,name))

# converting that dictionary to data frame
user_df = pd.DataFrame(user.values(),index=user.keys())
user_df.reset_index(inplace=True)
user_df.rename(columns={0:"Name","index":"id"},inplace=True)

# our work has done we get the data of users in this variable
# user_df

In [4]:
user_df

,id,Name
0,1,Saheba
1,2,Lokesh
2,3,Hemant
3,4,Abhishek
4,5,Kuldeep
5,6,Dev
6,7,Siddharth
7,8,Ajay
8,9,Sachin
9,10,Varun


**Now the data of prople and there ID are stored in user_data**

In [5]:
# because the data is in one column spliting it and keeping only the req. two columns so make these function that extract the data in first two columns

def get_id(text):
    return text.split("\t")[0]
def get_time(text):
    return text.split("\t")[1]

In [8]:
# creating two columns ID and punch_Time that hold the data when the employee punch in and out

df["id"]= df[0].map(get_id)
df["punch_time"] = df[0].map(get_time)
df.drop(columns=[0],inplace=True)   # deleting the raw data column and keeping only two column
df['id']= df["id"].astype(int)      # conerting id column to int

In [9]:
df.head(2)

,id,punch_time
0,27,2025-01-02 09:06:10
1,9,2025-01-02 09:07:36


In [10]:
# merge the data of id and punch time data into one data frame

data = pd.merge(df,user_df,how="left",on="id")

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484 entries, 0 to 483
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          484 non-null    int64 
 1   punch_time  484 non-null    object
 2   Name        484 non-null    object
dtypes: int64(1), object(2)
memory usage: 11.5+ KB


In [14]:
# extracting date and time

data['datetime'] = pd.to_datetime(df['punch_time'])
data['date'] = pd.to_datetime(df['punch_time']).dt.date
data['in_out_time'] = pd.to_datetime(df['punch_time']).dt.time
data["am_pm"] = data['datetime'].dt.strftime("%p")
data.drop(columns="punch_time",inplace=True)     # no use of this column now

In [27]:
dates = pd.date_range(start=data['datetime'].min().replace(day=1),end=data['datetime'].max())


In [35]:
dates = pd.DataFrame(dates,columns=['date'])

In [40]:
dates['date'] = dates['date'].dt.date

In [52]:
data = pd.merge(dates,data,how="left",on="date")
data['day_name'] = data['date'].dt.day_name()

AttributeError: Can only use .dt accessor with datetimelike values

In [51]:
data.head(2)

,date,id,Name,datetime,in_out_time,am_pm,day_name
0,2025-01-01,NaN,NaN,NaT,NaN,NaN,NaN
1,2025-01-02,27.0,Santosh,2025-01-02 09:06:10,09:06:10,AM,Thursday


In [ ]:
# we have only one column of IN and OUT time of employee , so making two columns to seperate them

ampm = data.pivot_table(index=data.index,columns='am_pm',values='datetime',aggfunc="min")
ampm.reset_index(inplace=True)
ampm.rename(columns={"index":"row_id"},inplace=True)
# ampm

In [ ]:
# merge both table into main data file

data = data.reset_index().rename(columns={"index":"row_id"})
data = pd.merge(data,ampm,how='inner',on ='row_id')

In [ ]:
# importing timesheet of employee IN and OUT time

timesheet = pd.read_excel("timesheet.xlsx",sheet_name="time")
ts = timesheet[['Name','max late','max early']]

In [ ]:
# merge time data to main dataframe

data = pd.merge(data,ts,how="inner",on ='Name')

In [ ]:
data['max late'] = data['max late'].dt.time
data['max early'] = data['max early'].dt.time
data['AM'] = data['AM'].dt.time
data['PM'] = data['PM'].dt.time

In [ ]:
# filling null values in AM column with max late and max early
data['AM'] = data['AM'].fillna(data['max late'])
data['PM'] = data['PM'].fillna(data['max early'])

In [ ]:
data['late_arival'] = data['AM']>data['max late']
data['early leave'] = data['PM']<data['max early']

In [ ]:
late_data = data.groupby("Name").agg({"late_arival":"sum","early leave":"sum"})

In [ ]:
# 'ta' stand for total attendance a temprary variable
ta = pd.DataFrame(data.groupby(["Name",'date'])['date'].count()).rename({"date":"count"},axis=1).reset_index()

In [ ]:
total_present = pd.DataFrame(ta.groupby("Name")['date'].count())

In [ ]:
final = pd.merge(total_present,late_data,on='Name',how='inner').reset_index()

In [ ]:
final.rename(columns={"date":"total_present_day"},inplace=True)

In [ ]:
final['net_present_day']=final['total_present_day']-(final['late_arival']*0.5 + final['early leave']*0.5)

In [ ]:
leave_data = pd.read_excel("leave_data.xlsx")

In [ ]:
leave_data = leave_data[['Name','Number of Days','Leave Start Date','Leave End Date']]

In [ ]:
leave_data['half_name'] = leave_data['Name'].map(lambda x : x.split()[0])

In [ ]:
# total leave taken by employees
total_leave = pd.DataFrame(leave_data.groupby('half_name')['Number of Days'].sum()).reset_index()

In [ ]:
# exporting total leave data to excel
total_leave.to_csv("Total_leave.csv")

In [ ]:
final = pd.merge(final,total_leave,left_on='Name',right_on="half_name",how="left")

In [ ]:
final.drop(columns="half_name",inplace=True)
final.rename({"Number of Days":"Leave"},axis=1,inplace=True)
final['Leave']= final['Leave'].fillna(0)
final.to_csv("FinalReport.csv")

In [ ]:
final

,Name,total_present_day,late_arival,early leave,net_present_day,Leave
0,Abhishek,25,0,0,25.0,0.0
1,Arvind,20,1,1,19.0,0.0
2,Devesh,24,0,2,23.0,0.0
3,Diksha,20,0,1,19.5,0.0
4,Kuldeep,24,0,0,24.0,0.0
5,Purnima,25,1,2,23.5,0.0
6,Sachin,8,0,4,6.0,0.0
7,Saheba,25,0,1,24.5,0.0
8,Santosh,23,2,1,21.5,0.0


In [ ]:
data.to_csv("Maindata.csv")

In [ ]:
user_df

,id,Name
0,1,Saheba
1,2,Lokesh
2,3,Hemant
3,4,Abhishek
4,5,Kuldeep
5,6,Dev
6,7,Siddharth
7,8,Ajay
8,9,Sachin
9,10,Varun


In [ ]:
df.to_csv("attendance csv file.csv")